This assignment uses the MNL model to analyze (1) yogurt purchase data made by consumers at a retail location, and (2) conjoint data about consumer preferences for minivans.


## 1. Estimating Yogurt Preferences

### Likelihood for the Multi-nomial Logit (MNL) Model

Suppose we have $i=1,\ldots,n$ consumers who each select exactly one product $j$ from a set of $J$ products. The outcome variable is the identity of the product chosen $y_i \in \{1, \ldots, J\}$ or equivalently a vector of $J-1$ zeros and $1$ one, where the $1$ indicates the selected product. For example, if the third product was chosen out of 4 products, then either $y=3$ or $y=(0,0,1,0)$ depending on how we want to represent it. Suppose also that we have a vector of data on each product $x_j$ (eg, size, price, etc.). 

We model the consumer's decision as the selection of the product that provides the most utility, and we'll specify the utility function as a linear function of the product characteristics:

$$ U_{ij} = x_j'\beta + \epsilon_{ij} $$

where $\epsilon_{ij}$ is an i.i.d. extreme value error term. 

The choice of the i.i.d. extreme value error term leads to a closed-form expression for the probability that consumer $i$ chooses product $j$:

$$ \mathbb{P}_i(j) = \frac{e^{x_j'\beta}}{\sum_{k=1}^Je^{x_k'\beta}} $$

For example, if there are 4 products, the probability that consumer $i$ chooses product 3 is:

$$ \mathbb{P}_i(3) = \frac{e^{x_3'\beta}}{e^{x_1'\beta} + e^{x_2'\beta} + e^{x_3'\beta} + e^{x_4'\beta}} $$

A clever way to write the individual likelihood function for consumer $i$ is the product of the $J$ probabilities, each raised to the power of an indicator variable ($\delta_{ij}$) that indicates the chosen product:

$$ L_i(\beta) = \prod_{j=1}^J \mathbb{P}_i(j)^{\delta_{ij}} = \mathbb{P}_i(1)^{\delta_{i1}} \times \ldots \times \mathbb{P}_i(J)^{\delta_{iJ}}$$

Notice that if the consumer selected product $j=3$, then $\delta_{i3}=1$ while $\delta_{i1}=\delta_{i2}=\delta_{i4}=0$ and the likelihood is:

$$ L_i(\beta) = \mathbb{P}_i(1)^0 \times \mathbb{P}_i(2)^0 \times \mathbb{P}_i(3)^1 \times \mathbb{P}_i(4)^0 = \mathbb{P}_i(3) = \frac{e^{x_3'\beta}}{\sum_{k=1}^Je^{x_k'\beta}} $$

The joint likelihood (across all consumers) is the product of the $n$ individual likelihoods:

$$ L_n(\beta) = \prod_{i=1}^n L_i(\beta) = \prod_{i=1}^n \prod_{j=1}^J \mathbb{P}_i(j)^{\delta_{ij}} $$

And the joint log-likelihood function is:

$$ \ell_n(\beta) = \sum_{i=1}^n \sum_{j=1}^J \delta_{ij} \log(\mathbb{P}_i(j)) $$


### Yogurt Dataset

We will use the `yogurt_data` dataset, which provides anonymized consumer identifiers (`id`), a vector indicating the chosen product (`y1`:`y4`), a vector indicating if any products were "featured" in the store as a form of advertising (`f1`:`f4`), and the products' prices (`p1`:`p4`). For example, consumer 1 purchased yogurt 4 at a price of 0.079/oz and none of the yogurts were featured/advertised at the time of consumer 1's purchase.  Consumers 2 through 7 each bought yogurt 2, etc.


In [ ]:
#| echo: false

import pandas as pd
import numpy as np

In [ ]:
#| echo: false

yogurt = pd.read_csv("yogurt_data.csv")
yogurt

The dataset contains the following columns:

*   `id`: Unique identifier for each consumer.
*   `y1` to `y4`: Binary indicators showing which of the four yogurts was chosen by the consumer. Only one of these columns will have a value of `1` per row, indicating the chosen product.
*   `f1` to `f4`: Binary indicators showing whether each of the four yogurts was featured (advertised) in the store at the time of the purchase.
*   `p1` to `p4`: Numeric values representing the price per ounce of each of the four yogurts.

From the preview of the data:

*   In the first row, consumer with `id` 1 purchased yogurt 4 (as indicated by `y4` = 1) at a price of 0.079 per ounce. None of the yogurts were featured.
*   Subsequent rows show similar data for other consumers, indicating their choices and the conditions (price and whether the product was featured) under each product.


Let the vector of product features include brand dummy variables for yogurts 1-3 (we'll omit a dummy for product 4 to avoid multi-collinearity), a dummy variable to indicate if a yogurt was featured, and a continuous variable for the yogurts' prices:  

$$ x_j' = [\mathbbm{1}(\text{Yogurt 1}), \mathbbm{1}(\text{Yogurt 2}), \mathbbm{1}(\text{Yogurt 3}), X_f, X_p] $$

The "hard part" of the MNL likelihood function is organizing the data, as we need to keep track of 3 dimensions (consumer $i$, covariate $k$, and product $j$) instead of the typical 2 dimensions for cross-sectional regression models (consumer $i$ and covariate $k$). 

What we would like to do is reorganize the data from a "wide" shape with $n$ rows and multiple columns for each covariate, to a "long" shape with $n \times J$ rows and a single column for each covariate.  As part of this re-organization, we'll add binary variables to indicate the first 3 products; the variables for featured and price are included in the dataset and simply need to be "pivoted" or "melted" from wide to long.  

Let's reshape the dataset for analysis:


In [ ]:
#| echo: false

# Define the columns corresponding to choices, features, and prices
choice_columns = [f'y{j}' for j in range(1, 5)]
feature_columns = [f'f{j}' for j in range(1, 5)]
price_columns = [f'p{j}' for j in range(1, 5)]

In [ ]:
#| echo: false

# Convert wide format to long format using the correct dataset name "yogurt"
long_data_final = pd.DataFrame()

# Loop through each product to create the long format data
for j in range(1, 5):
    temp_df = yogurt[['id', f'y{j}', f'f{j}', f'p{j}']].copy()
    temp_df.columns = ['id', 'choice', 'featured', 'price']
    temp_df['product'] = j
    long_data_final = pd.concat([long_data_final, temp_df], ignore_index=True)

In [ ]:
#| echo: false

# Adding dummy variables for the first three products
long_data_final['brand1'] = (long_data_final['product'] == 1).astype(int)
long_data_final['brand2'] = (long_data_final['product'] == 2).astype(int)
long_data_final['brand3'] = (long_data_final['product'] == 3).astype(int)

# Display the reshaped data
long_data_final.head()

The dataset has been reshaped into the long format using the dataset named "yogurt". Here is the structure of the transformed data:

*   `id`: Consumer identifier.
*   `choice`: Indicates whether the consumer chose this product (1 if chosen, 0 otherwise).
*   `featured`: Indicates whether the product was featured (1 if featured, 0 otherwise).
*   `price`: Price per ounce of the product.
*   `product`: Product number (1 through 4).
*   `brand1`, `brand2`, `brand3`: Dummy variables for the first three products, with the fourth product omitted to avoid multicollinearity.


### Estimation


1.  **Utility Function**: Calculates the utility for each product based on the beta coefficients and the features in the dataset.
2.  **Probability Calculation**: Determines the probabilities of choosing each product by a consumer, factoring in the utilities of all available products for normalization.
3.  **Log-Likelihood**: Computes the log of these probabilities where the product was actually chosen (where `choice == 1`) and sums these values to obtain the log-likelihood for the dataset given the model parameters.


In [ ]:
#| echo: false

import numpy as np
import pandas as pd

# Defining the utility function
def utility_function(data, beta):
    # Utility is a linear combination of the features
    # beta: [beta_brand1, beta_brand2, beta_brand3, beta_featured, beta_price]
    utility = (data['brand1'] * beta[0] +
               data['brand2'] * beta[1] +
               data['brand3'] * beta[2] +
               data['featured'] * beta[3] +
               data['price'] * beta[4])
    return utility

# Function to calculate the probabilities
def calculate_probabilities(data, beta):
    data['utility'] = utility_function(data, beta)
    # Calculating the exponential of utilities for each consumer
    max_util = data.groupby('id')['utility'].transform('max')  # Stability improvement
    data['exp_utilities'] = np.exp(data['utility'] - max_util)  # Subtracting max for numerical stability
    sum_exp_utilities = data.groupby('id')['exp_utilities'].transform('sum')
    data['probability'] = data['exp_utilities'] / sum_exp_utilities
    return data

# Function to calculate the log-likelihood
def log_likelihood(data, beta):
    data = calculate_probabilities(data, beta)
    # Only taking logs of probabilities where choice is 1
    log_probs = np.log(data.loc[data['choice'] == 1, 'probability'])
    return log_probs.sum()

# Example usage with hypothetical beta coefficients
beta_example = [0.5, -0.3, 0.2, 0.1, -0.05]
# Assuming 'long_data_final' is your DataFrame after reshaping and adding dummy variables
log_likelihood_value = log_likelihood(long_data_final, beta_example)
print("Log-Likelihood Value:", log_likelihood_value)

Optimizin gthe model to find the MLEs for the 5 parameters ($\beta_1, \beta_2, \beta_3, \beta_f, \beta_p$).  (Hint: you should find 2 positive and 1 negative product intercepts, a small positive coefficient estimate for featured, and a large negative coefficient estimate for price.)


In [ ]:
#| echo: false

from scipy.optimize import minimize

# Define the negative log-likelihood function
def negative_log_likelihood(beta, data):
    return -log_likelihood(data, beta)

# Initial guess for beta coefficients
initial_beta = [0.1, 0.1, 0.1, 0.1, -0.1]

# Define bounds for the parameters
bounds = [(None, None), (None, None), (None, None), (0, None), (None, 0)]

In [ ]:
#| echo: true

# Optimization to find the MLE of the beta coefficients
result = minimize(negative_log_likelihood, initial_beta, args=(long_data_final,), 
                  method='L-BFGS-B', bounds=bounds)

# Print the optimization results
print("Optimization Success:", result.success)
print("Estimated beta coefficients:", result.x)

### Discussion

We learn...

From the three product intercepts from the estimated beta coefficients for Brands 1, 2, and 3, we can gauge consumer preferences among the yogurt options relative to the baseline (Brand 4, which does not have a specific intercept and is implied as zero in the model). Here’s how to interpret the intercepts:

1.  **Beta for Brand 1 (`beta_1`)**: 1.388
    
    *   This positive intercept suggests that, holding other factors constant (such as price and whether the product was featured), Brand 1 is preferred over Brand 4. The higher positive value indicates a strong preference for this brand.
2.  **Beta for Brand 2 (`beta_2`)**: 0.644
    
    *   Similarly, this positive intercept indicates a preference for Brand 2 over Brand 4, though the preference is less strong compared to Brand 1. Brand 2 is still preferred but not as much as Brand 1.
3.  **Beta for Brand 3 (`beta_3`)**: −3.086
    
    *   The negative value of this intercept indicates that Brand 3 is less preferred compared to Brand 4. This substantial negative value suggests a strong aversion to Brand 3 relative to the other options.

### Which yogurt is most preferred?

Based on the magnitude and direction of the beta coefficients:

*   **Brand 1** is the most preferred yogurt among the options. It has the highest positive intercept, indicating the strongest preference relative to the baseline (Brand 4).
*   **Brand 2** is also preferred over the baseline but to a lesser extent than Brand 1.
*   **Brand 3** is the least preferred, with a significant negative intercept indicating a strong preference against it compared to Brand 4.

This interpretation can help in strategic decisions related to marketing and product placement, such as focusing promotional efforts on Brands 1 and 2, considering price adjustments or rebranding for Brand 3, and understanding the competitive position of Brand 4 as a potentially neutral or fallback option.


*   **Identify the most and least preferred yogurts**:
    
    *   Most preferred: Brand 1 (intercept = 1.388)
    *   Least preferred: Brand 3 (intercept = -3.086)
*   **Calculate the difference in utilities** between the most preferred and least preferred yogurts:
    
*   **Use the price coefficient as a conversion factor**:
    
    *   Price coefficient (`beta_p`): −37.058
    *   The price coefficient's magnitude indicates the decrease in the utility of purchasing a yogurt per dollar increase in its price. Therefore, it acts as a conversion factor from utility to dollar value.


In [ ]:
#| echo: false

# Calculated difference in utility from the coefficients for Brand 1 and Brand 3
utility_difference = 1.388 - (-3.086)

# Price coefficient
price_coefficient = -37.058

# Convert utility to dollar value using the price coefficient
dollar_value_per_util = 1 / price_coefficient

In [ ]:
#| echo: true

# Calculate the dollar benefit
dollar_benefit = utility_difference * dollar_value_per_util
dollar_benefit

The calculated dollar benefit between the most preferred yogurt (Brand 1) and the least preferred yogurt (Brand 3) is approximately \-$0.121 per unit. This indicates that consumers might be willing to pay about 12 cents more per unit for Brand 1 over Brand 3 due to their difference in utility, based on the model estimates.

This negative sign might initially seem counterintuitive, but it reflects how much less consumers would be willing to pay for the less preferred option (Brand 3). In other words, Brand 1 has a higher inherent value of about 12 cents per unit over Brand 3 in the eyes of consumers.

This monetary measure effectively quantifies the brand value in dollar terms based on consumer preferences modeled by the MNL. This insight can be leveraged for pricing strategies and marketing efforts to enhance product positioning and profitability.

One benefit of the MNL model is that we can simulate counterfactuals (eg, what if the price of yogurt 1 was $0.10/oz instead of $0.08/oz).


To calculate the market shares and predict the changes in market share due to a price increase for Yogurt 1,

*   **Calculate Current Market Shares**:
    
    *   Count the number of times each product was chosen and divide by the total number of observations.
*   **Simulate the Effect of a Price Increase for Yogurt 1**:
    
    *   Increase the price of Yogurt 1 by $0.10 and use the fitted model to estimate the choice probabilities for each product and each consumer.
    *   Calculate the expected market shares by taking the average of the choice probabilities across all consumers for each product.


In [ ]:
#| echo: true

# Calculate the current market shares
current_market_shares = long_data_final.groupby('product')['choice'].mean()
print("Current Market Shares:", current_market_shares)

In [ ]:
#| echo: false

# Function to update the price of Yogurt 1 and calculate new probabilities
def predict_new_market_shares(data, beta, price_increase=0.10):
    # Create a copy to avoid modifying the original data
    modified_data = data.copy()
    
    # Apply price increase to Yogurt 1
    modified_data.loc[modified_data['product'] == 1, 'price'] += price_increase
    
    # Calculate new choice probabilities
    modified_data = calculate_probabilities(modified_data, beta)
    
    # Calculate new market shares by averaging choice probabilities by product
    new_market_shares = modified_data.groupby('product')['probability'].mean()
    return new_market_shares

In [ ]:
#| echo: true

# Call the function with the original data and fitted beta coefficients
new_market_shares = predict_new_market_shares(long_data_final, result.x)
print("New Market Shares After Price Increase:", new_market_shares)

Based on the results provided:

### Current Market Shares:

*   **Yogurt 1**: 34.2%
*   **Yogurt 2**: 40.1%
*   **Yogurt 3**: 2.9%
*   **Yogurt 4**: 22.8%

### New Market Shares After a $0.10 Price Increase to Yogurt 1:

*   **Yogurt 1**: 2.1%
*   **Yogurt 2**: 59.1%
*   **Yogurt 3**: 4.4%
*   **Yogurt 4**: 34.4%

### Analysis:

*   **Yogurt 1 Market Share**: The market share for Yogurt 1 drastically decreases from 34.2% to 2.1%. This significant reduction clearly indicates that the price sensitivity for Yogurt 1 is quite high, and even a modest price increase leads to a substantial shift in consumer preferences away from this product.
*   **Shifts to Other Products**: The market shares of Yogurt 2, 3, and 4 all increase. Yogurt 2 benefits the most, with its market share increasing by nearly 19 percentage points. This suggests that consumers consider Yogurt 2 a close substitute for Yogurt 1, switching to it when Yogurt 1 becomes more expensive.
*   **Yogurt 3 and 4**: Even though Yogurt 3 had a very low initial market share, it sees a relative increase, suggesting some consumers might switch to lower preference options if their first choice becomes less economically viable. Yogurt 4 also sees a notable increase in market share.

### Conclusion:

The $0.10 price increase for Yogurt 1 drastically reduces its market share, confirming the price sensitivity and showing a clear preference shift among consumers to other available options, particularly Yogurt 2. This demonstrates the importance of pricing strategies in competitive markets and highlights the value of using such models for predictive analytics in marketing and pricing decisions.


## 2. Estimating Minivan Preferences


### Data


In [ ]:
#| echo: false

import statsmodels.api as sm

conjoint_data = pd.read_csv("rintro-chapter13conjoint.csv")
conjoint_data 

The dataset `rintro-chapter13conjoint.csv` appears to contain data from a conjoint analysis, typically used to gauge consumer preferences across different product features. Here's a breakdown of the columns:

*   `resp.id`: Identifier for respondents.
*   `ques`: Question or scenario number.
*   `alt`: Alternative options presented within each question.
*   `carpool`: Indicates if the option includes a carpool feature (yes/no).
*   `seat`: Number of seats.
*   `cargo`: Cargo space.
*   `eng`: Type of engine (e.g., 'gas' for gasoline, 'hyb' for hybrid).
*   `price`: Price in thousands of dollars.
*   `choice`: Binary indicator of whether the respondent chose this alternative (1 for chosen, 0 for not chosen).


In [ ]:
#| echo: false

# Number of unique respondents
num_respondents = conjoint_data['resp.id'].nunique()

# Number of choice tasks per respondent
tasks_per_respondent = conjoint_data.groupby('resp.id')['ques'].nunique()

# Number of alternatives per choice task (assuming all respondents faced the same task structure)
alternatives_per_task = conjoint_data.groupby(['resp.id', 'ques'])['alt'].nunique().mode()[0]

In [ ]:
#| echo: true

num_respondents, tasks_per_respondent.describe(), alternatives_per_task

Here's a summary of the dataset based on the analysis:

1.  **Number of Respondents**: There are 200 unique respondents who participated in the conjoint survey.
    
2.  **Choice Tasks Per Respondent**:
    
    *   Each respondent completed 15 choice tasks.
    *   This is consistent across all respondents, as indicated by the lack of variability (standard deviation is 0).
3.  **Alternatives Presented in Each Choice Task**:
    
    *   Each choice task presented 3 alternatives to the respondents.
    *   This number is consistent across all tasks and respondents, as shown by the uniform count and lack of variability.


The attributes (levels) were number of seats (6,7,8), cargo space (2ft, 3ft), engine type (gas, hybrid, electric), and price (in thousands of dollars).


### Model

Estimating a MNL model omitting the following levels to avoide multicollinearity (6 seats, 2ft cargo, and gas engine). Include price as a continuous variable. Show a table of coefficients and standard errors.  You may use your own likelihood function from above, or you may use a function from a package/library to perform the estimation.


In [ ]:
#| echo: false

conjoint_data['carpool'] = conjoint_data['carpool'].map({'yes': 1, 'no': 0})
conjoint_data = pd.get_dummies(conjoint_data, columns=['seat', 'cargo', 'eng'], drop_first=True)

# Convert boolean columns to integer type
conjoint_data['seat_7'] = conjoint_data['seat_7'].astype(int)
conjoint_data['seat_8'] = conjoint_data['seat_8'].astype(int)
conjoint_data['cargo_3ft'] = conjoint_data['cargo_3ft'].astype(int)
conjoint_data['eng_hyb'] = conjoint_data['eng_hyb'].astype(int)

# Ensure all data is numeric and handle any NaN values appropriately
conjoint_data.fillna(0, inplace=True)  # Example: Fill NA/NaN values with zero
conjoint_data = conjoint_data.apply(pd.to_numeric, errors='coerce')  # Coerce errors will convert non-convertible terms to NaN

# Define the features and the target variable
X = conjoint_data[['carpool', 'price', 'seat_7', 'seat_8', 'cargo_3ft', 'eng_hyb']]
y = conjoint_data['choice']

In [ ]:
#| echo: true

# Adding a constant to the model (intercept)
X = sm.add_constant(X)

# Check if there are still any object types
print(X.dtypes, y.dtypes)


In [ ]:
#| echo: true

# Fit the MNL model
mnl_model = sm.MNLogit(y, X)
mnl_result = mnl_model.fit()

# Print the summary of the model
print(mnl_result.summary())

### Results

*   **Price**: The negative coefficient for price (−0.1486) indicates that as the price increases, the likelihood of choosing that option decreases, as expected.
*   **Seat 7 and Seat 8**: Both coefficients are negative, indicating less preference compared to the baseline (6 seats). The magnitude of the coefficient for Seat 7 is larger, suggesting a stronger negative preference compared to Seat 8.
*   **Cargo 3**: The positive coefficient (0.4099) suggests a preference for more cargo space (3ft) over the baseline (2ft).
*   **Eng Hyb**: The slight negative coefficient for the hybrid engine (−0.1102) suggests a slight aversion compared to the gasoline engine, although this effect is relatively small.

These coefficients provide insights into the preferences of respondents regarding different features of the products presented in the conjoint study. The standard errors indicate the precision of these estimates, with all p-values being significant, showing strong evidence against the null hypothesis of no effect.


Using the price coefficient as a dollar-per-util conversion factor. What is the dollar value of 3ft of cargo space as compared to 2ft of cargo space?


In [ ]:
#| echo: true

# Coefficients from the model
cargo_3ft_coeff = mnl_result.params.loc['cargo_3ft'][0]
price_coeff = mnl_result.params.loc['price'][0]

# Calculate the dollar value of 3ft of cargo space compared to 2ft
dollar_value_cargo_3ft = cargo_3ft_coeff / -price_coeff
dollar_value_cargo_3ft

The dollar value of having 3ft of cargo space as compared to 2ft, based on the estimated model coefficients, is approximately **$2.76**.

This means that, according to the model, consumers would be willing to pay an additional $2.76 for a vehicle option that offers 3ft of cargo space over one that offers only 2ft, all else being equal. This calculation converts the utility gained from larger cargo space into a monetary equivalent using the sensitivity of consumer choices to changes in price.


Assuming the market consists of the following 6 minivans. Predict the market shares of each minivan in the market.

| Minivan | Seats | Cargo | Engine | Price |
|---------|-------|-------|--------|-------|
| A       | 7     | 2     | Hyb    | 30    |
| B       | 6     | 2     | Gas    | 30    |
| C       | 8     | 2     | Gas    | 30    |
| D       | 7     | 3     | Gas    | 40    |
| E       | 6     | 2     | Elec   | 40    |
| F       | 7     | 2     | Hyb    | 35    |


In [ ]:
#| echo: false


import numpy as np

# Define the minivan market configuration
minivans = pd.DataFrame({
    'Minivan': ['A', 'B', 'C', 'D', 'E', 'F'],
    'Seats': [7, 6, 8, 7, 6, 7],
    'Cargo': [2, 2, 2, 3, 2, 2],
    'Engine': ['Hyb', 'Gas', 'Gas', 'Gas', 'Elec', 'Hyb'],
    'Price': [30, 30, 30, 40, 40, 35]
})

# Convert categorical data to the model format (one-hot encoding)
minivans['seat_7'] = (minivans['Seats'] == 7).astype(int)
minivans['seat_8'] = (minivans['Seats'] == 8).astype(int)
minivans['cargo_3ft'] = (minivans['Cargo'] == 3).astype(int)
minivans['eng_hyb'] = (minivans['Engine'] == 'Hyb').astype(int)
minivans['eng_elec'] = (minivans['Engine'] == 'Elec').astype(int)

# Setup the features for utility calculation
X_minivans = minivans[['seat_7', 'seat_8', 'cargo_3ft', 'eng_hyb', 'eng_elec', 'Price']]
X_minivans = sm.add_constant(X_minivans)  # add constant for the intercept

In [ ]:
#| echo: true


# Calculate utilities for each minivan
utilities = np.dot(X_minivans, mnl_result.params.values)
# Compute market shares using softmax function
market_shares = np.exp(utilities) / np.sum(np.exp(utilities), axis=0)

# Add market shares to the minivans dataframe
minivans['Market Share'] = market_shares
minivans[['Minivan', 'Market Share']]

### Market Share Interpretation

1.  **Minivan B (26.40%)**:
    
    *   **Attributes**: 6 seats, 2ft cargo, Gas engine, priced at $30.
    *   **Interpretation**: This option has the highest market share, which suggests that a lower price combined with the base features (6 seats, Gas engine) is highly attractive to consumers. The absence of additional features seems to be offset by the lower price.
2.  **Minivan C (22.76%)**:
    
    *   **Attributes**: 8 seats, 2ft cargo, Gas engine, also priced at $30.
    *   **Interpretation**: The second most popular option, indicating a strong preference for more seating capacity at no additional cost in price, suggesting that consumers value additional seating when all other factors are equal.
3.  **Minivan A (20.42%)**:
    
    *   **Attributes**: 7 seats, 2ft cargo, Hybrid engine, priced at $30.
    *   **Interpretation**: The presence of a Hybrid engine with moderate pricing and seat count shows significant consumer interest in more environmentally friendly options when they are not priced higher than gas models.
4.  **Minivan E (13.21%)**:
    
    *   **Attributes**: 6 seats, 2ft cargo, Electric engine, priced at $40.
    *   **Interpretation**: Indicates a moderate interest in electric vehicles, though the higher price seems to be a deterrent compared to cheaper gas options.
5.  **Minivan F (11.77%)**:
    
    *   **Attributes**: 7 seats, 2ft cargo, Hybrid engine, priced at $35.
    *   **Interpretation**: Similar to Minivan A but slightly more expensive, which slightly reduces its attractiveness despite the hybrid engine.
6.  **Minivan D (5.45%)**:
    
    *   **Attributes**: 7 seats, 3ft cargo, Gas engine, priced at $40.
    *   **Interpretation**: Despite offering more cargo space, its higher price significantly lowers its market share. This suggests that price sensitivity may outweigh the benefit of additional cargo space for many consumers.

### Overall Insights

*   **Price Sensitivity**: Price plays a crucial role in consumer choices, as seen by the higher market shares of the lower-priced models.
*   **Preference for Seat Capacity**: There is a clear preference for higher seating capacity when the price is held constant.
*   **Eco-friendly Options**: Consumers show interest in hybrid and electric vehicles, but their choices are sensitive to price differences.
*   **Cargo Space Value**: Additional cargo space does not seem to drive higher market share, especially when accompanied by a higher price.

These insights could be used to strategize product offerings, pricing decisions, and marketing campaigns by highlighting features that align with consumer preferences.


_hint: this example is taken from the "R 4 Marketing Research" book by Chapman and Feit. I believe the same example is present in the companion book titled "Python 4 Marketing Research".  I encourage you to attempt these questions on your own, but if you get stuck or would like to compare you results to "the answers," you may consult the Chapman and Feit books._
